<a href="https://colab.research.google.com/github/timduignan/orb-models/blob/main/examples/OrbMDTut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First you need to enable the gpu under Runtime, change runtime type. Then pip install the relevant libraries:

In [1]:
!pip install -q orb-models
!pip install -q ase
!pip install -q "pynanoflann@git+https://github.com/dwastberg/pynanoflann#egg=af434039ae14bedcbb838a7808924d6689274168"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


Then we need to get the example input file and run script, which are in the github repo

In [2]:
!wget https://raw.githubusercontent.com/orbital-materials/orb-models/main/examples/NaClWater.xyz
!wget https://raw.githubusercontent.com/orbital-materials/orb-models/main/examples/NaClWaterMD.py

--2025-01-24 00:53:07--  https://raw.githubusercontent.com/orbital-materials/orb-models/main/examples/NaClWater.xyz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82391 (80K) [text/plain]
Saving to: ‘NaClWater.xyz’

NaClWater.xyz       100%[===================>]  80.46K  --.-KB/s    in 0.001s  

2025-01-24 00:53:08 (55.9 MB/s) - ‘NaClWater.xyz’ saved [82391/82391]

--2025-01-24 00:53:08--  https://raw.githubusercontent.com/orbital-materials/orb-models/main/examples/NaClWaterMD.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24

In [3]:
!ls

NaClWaterMD.py	NaClWater.xyz  sample_data


Then simply run the python script.

In [4]:
!python NaClWaterMD.py

Using device: cuda
/usr/local/lib/python3.11/dist-packages/orb_models/forcefield/pretrained.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(loca

It has generated 100 frames of simulation we can see it looks to be stable at 300 K

In [5]:
!head md_nvt.log

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000         -7644.17     -7705.77        61.60   300.9
0.0005         -7644.08     -7705.80        61.72   301.5
0.0010         -7644.13     -7705.90        61.77   301.7
0.0015         -7644.17     -7706.02        61.85   302.1
0.0020         -7644.16     -7706.05        61.89   302.3
0.0025         -7644.05     -7705.92        61.87   302.2
0.0030         -7644.18     -7705.64        61.46   300.2
0.0035         -7644.15     -7705.25        61.11   298.4
0.0040         -7644.01     -7704.82        60.81   297.0


The trajectory is stored in NaClWaterMD.xyz. We can visualise a frame of the simulation:

In [6]:
pip install py3Dmol

In [7]:
import py3Dmol

# Function to extract a specific frame from the .xyz file
def extract_frame(xyz_file, frame_number):
    with open(xyz_file, "r") as f:
        lines = f.readlines()

    # Parse the file
    num_atoms = int(lines[0].strip())  # Number of atoms from the first line
    frame_start = frame_number * (num_atoms + 2)  # Start of the desired frame
    frame_lines = lines[frame_start : frame_start + num_atoms + 2]  # Extract frame
    return "".join(frame_lines)


# Specify the file and the frame number
frame_number = 3  # Pick the frame (0-based indexing)
xyz_file = "NaClWaterMD.xyz"
frame_data = extract_frame(xyz_file, frame_number)

# Visualize the extracted frame with balls and sticks
viewer = py3Dmol.view(width=800, height=400)
viewer.addModel(frame_data, "xyz")  # Add the trajectory frame
viewer.setStyle({
    "stick": {},                      # Default stick style
    "sphere": {"radius": 0.5}        # Sphere style with adjustable radius
})
viewer.zoomTo()
viewer.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Then you can run it long at a high temperatureat to see how stable it is. Or upload your own xyz file and run that just remember to update the cell size parameter and reduce the print frequency.

In [ ]:
from NaClWaterMD import run_md_simulation

# Run with custom parameters
run_md_simulation(
    input_file="NaClWater.xyz",
    cell_size= 25.25,
    temperature_K=400,
    total_steps=10000,
    traj_interval = 20,
    log_interval=20
)

This script should also work on the gpu on a silicon Mac and is should be fast enough for many purposese. You probably want to load a conda environment or similar with

```
conda create --name orb_env python=3.11
conda activate orb_env

```

Please reach out if you have any questions or issues. My email is tim@orbitalmaterials.com
Happy simulating!